<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/computervision/FastSAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
!git clone https://huggingface.co/spaces/An-619/FastSAM

In [ ]:
%cd /content/FastSAM
!pip install -r requirements.txt
!pip install gradio

Start WebUI

In [ ]:
%cd /content/FastSAM
!python app.py

stable diffusion with background

In [ ]:
%cd /content
!git clone https://github.com/SysCV/sam-hq.git

In [ ]:
%cd /content
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

print("PyTorch version:", torch.__version__)
print("CUDA is available:", torch.cuda.is_available())

os.chdir('sam-hq')
!export PYTHONPATH=$(pwd)
from segment_anything import sam_model_registry, SamPredictor

In [ ]:
!mkdir pretrained_checkpoint
!wget https://huggingface.co/lkeab/hq-sam/resolve/main/sam_hq_vit_l.pth
!mv sam_hq_vit_l.pth pretrained_checkpoint

In [2]:
# utils function
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))


def show_res(masks, scores, input_point, input_label, input_box, image):
    for i, (mask, score) in enumerate(zip(masks, scores)):
        plt.figure(figsize=(10,10))
        plt.imshow(image)
        show_mask(mask, plt.gca())
        if input_box is not None:
            box = input_box[i]
            show_box(box, plt.gca())
        if (input_point is not None) and (input_label is not None):
            show_points(input_point, input_label, plt.gca())

        print(f"Score: {score:.3f}")
        plt.axis('off')
        plt.show()


def show_res_multi(masks, scores, input_point, input_label, input_box, image):
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    for mask in masks:
        show_mask(mask, plt.gca(), random_color=True)
    for box in input_box:
        show_box(box, plt.gca())
    for score in scores:
        print(f"Score: {score:.3f}")
    plt.axis('off')
    plt.show()


In [ ]:
sam_checkpoint = "pretrained_checkpoint/sam_hq_vit_l.pth"
model_type = "vit_l"
device = "cuda"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)

In [ ]:
image = cv2.imread('/content/anya.jpeg')
print(image.shape)

In [ ]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([[5,5,180,250]])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,
    hq_token_only= False,
)
show_res(masks,scores,input_point, input_label, input_box, image)

In [ ]:
image = cv2.imread('/content/anya.jpeg')
input_point = np.array([[95,40],[165, 55], [30, 55], [50,230],[140, 230]])
input_label = np.ones(input_point.shape[0])
input_box = None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,
    hq_token_only= True,
)
show_res(masks,scores,input_point, input_label, input_box, image)

In [ ]:
from PIL import Image
for mask in masks:
  h, w = mask.shape[-2:]
  display(Image.fromarray(~mask))
  Image.fromarray(~mask).save("/content/anya_mask.jpg")

Background CivitAI

In [ ]:
# 7536
!wget https://civitai.com/api/download/models/7536 -O /content/background.safetensors

In [ ]:
!pip install diffusers accelerate bitsandbytes transformers

In [8]:
from diffusers import StableDiffusionInpaintPipeline
import torch

In [ ]:
model_name = "stablediffusionapi/anything-v5"
pipe = StableDiffusionInpaintPipeline.from_pretrained(model_name, torch_dtype=torch.float16)
pipe.to("cuda")

In [28]:
from PIL import Image
init_image = Image.open("/content/anya.jpeg").resize((512, 512))
mask_image = Image.open("/content/anya_mask.jpg").resize((512, 512))
bw, bh = Image.open("/content/anya.jpeg").size

In [ ]:
prompt = "wedding party"
image = pipe(prompt=prompt, image=init_image, mask_image=mask_image, num_inference_steps=30).images[0]
display(image.resize((bw, bh)))